In [2]:
! pip install datasets
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch

from datasets import load_dataset
from nltk.tokenize import ToktokTokenizer
from sklearn.metrics import f1_score
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader


In [47]:
from IPython.core.display import TextDisplayObject


In [18]:
from sklearn.metrics import f1_score


In [3]:
!wget  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
!unzip  "wiki-news-300d-1M.vec.zip" -d "."

--2022-12-28 22:33:15--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  37.1MB/s    in 18s     

2022-12-28 22:33:34 (35.6 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]

Archive:  wiki-news-300d-1M.vec.zip
  inflating: ./wiki-news-300d-1M.vec  


In [4]:
def load_embeddings(file_path, pad_token='PAD', unk_token='UNK', max_words=100_000, verbose=True):
    
    vocab = dict()
    embeddings = list()

    with open(file_path) as file_object:

        vocab_size, embedding_dim = file_object.readline().strip().split()

        vocab_size = int(vocab_size)
        embedding_dim = int(embedding_dim)

        # в файле 1 000 000 слов с векторами, давайте ограничим для простоты этот словарь
        max_words = vocab_size if max_words <= 0 else max_words
        vocab: Dict[str, int] = dict()
        # добавим пад токен и эмбеддинг в нашу матрицу эмбеддингов и словарь
        vocab[pad_token] = 0
        # добавим унк токен и эмбеддинг в нашу матрицу эмбеддингов и словарь
        vocab[unk_token] = 1
        
        embeddings = np.zeros((max_words, embedding_dim))
        embeddings[1,:] = (np.ones(embedding_dim))

        progress_bar = tqdm(total=max_words, disable=not verbose, desc='Reading embeddings file')

        for line in file_object:
            parts = line.strip().split()

            token = ' '.join(parts[:-embedding_dim]).lower()

            if token in vocab:
                continue

            vocab[token] = len(vocab)

            embeddings[vocab[token], :] = np.array(list(map(float, parts[-embedding_dim:])))

            progress_bar.update()

            if len(vocab) == max_words:
                break

        progress_bar.close()

    assert(len(vocab) == embeddings.shape[0])

    #embeddings = np.stack(embeddings)
    
    return vocab, embeddings

In [5]:
vocab, embeddings = load_embeddings('wiki-news-300d-1M.vec', max_words=100_000)

Reading embeddings file: 100%|█████████▉| 99998/100000 [00:08<00:00, 11246.45it/s]


In [6]:
dataset_path = "tweet_eval"
dataset_name = "sentiment"

train_dataset = load_dataset(path=dataset_path, name=dataset_name, split="train")
valid_dataset = load_dataset(path=dataset_path, name=dataset_name, split="validation")
test_dataset = load_dataset(path=dataset_path, name=dataset_name, split="test")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


In [7]:
def empty_collate(batch):
    return batch

In [8]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=empty_collate)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=True, collate_fn=empty_collate)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True, collate_fn=empty_collate)

In [9]:
class Tokenizer:
    
    def __init__(self, base_tokenizer, token2index, pad_token, unk_token, max_length):
        
        self._base_tokenizer = base_tokenizer  
        self.token2index = token2index 
        self.pad_token = pad_token
        self.pad_index = self.token2index[self.pad_token]    
        self.unk_token = unk_token
        self.unk_index = self.token2index[self.unk_token]
        
        self.max_length = max_length

    def tokenize(self, text):
        """
        В этом методе нужно разделить строку текста на токены
        """
        return self._base_tokenizer.tokenize(text)
    
    def indexing(self, tokenized_text):
        """
        В этом методе нужно перевести список токенов в список с индексами этих токенов
        """
        tokens_indices = []
        for t in tokenized_text:
          if t in self.token2index:
            tokens_indices.append(self.token2index[t])
          else:
            tokens_indices.append(self.unk_index)

        return tokens_indices
        
    def padding(self, tokens_indices):
        """
        В этом методе нужно сделать длину tokens_indices равной self.max_length
        Опционально убрать повторяющиеся unk'и
        """
        if len(tokens_indices) > self.max_length:
          return tokens_indices[:self.max_length]
        elif len(tokens_indices) < self.max_length:
          return tokens_indices + [self.pad_index for i in range(self.max_length - len(tokens_indices))]
        
    
    def __call__(self, text):
        """
        В этом методе нужно перевести строку с текстом в вектор с индексами слов нужно размера (self.max_length)
        """
        return self.padding(self.indexing(self.tokenize(text)))
        
    def collate(self, batch):
        
        tokenized_texts = list()
        labels = list()
        
        for sample in batch:
            tokenized_texts.append(Tokenizer(base_tokenizer=self._base_tokenizer, token2index=self.token2index, pad_token=self.pad_token, unk_token=self.unk_token, max_length=self.max_length)(sample['text']))
            labels.append(sample['label'])
            
        tokenized_texts = torch.Tensor(tokenized_texts).to(torch.int64)  # перевод в torch.Tensor
        labels = torch.Tensor(labels).to(torch.int64)  # перевод в torch.Tensor
        

        return tokenized_texts, labels

In [10]:
tokenizer = Tokenizer(base_tokenizer=ToktokTokenizer(), token2index=vocab, unk_token='UNK', pad_token='PAD', max_length=64)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=tokenizer.collate)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False, collate_fn=tokenizer.collate)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, collate_fn=tokenizer.collate)


In [12]:
for x, y in train_loader:
    break

In [44]:
class DeepAverageNetwork(nn.Module):
    
    def __init__(self,
                 n_embed: int,
                 d_embed: int,
                 d_hidden: int,
                 d_out: int,
                 dp: float,
                 embed_weight:np.array):
        super(DeepAverageNetwork, self).__init__()


        self.embed = nn.Embedding(n_embed, d_embed)

 
        self.dropout1 = nn.Dropout(dp)

        self.bn1 = nn.BatchNorm1d(d_embed)

        self.fc1 = nn.Linear(d_embed, d_hidden)

        self.non_linear_function = torch.nn.ReLU()




        self.dropout2 = nn.Dropout(dp)

        self.bn2 = nn.BatchNorm1d(d_hidden)

        self.fc2 = nn.Linear(d_hidden, d_out)

        self.non_linear_function = torch.nn.ReLU()




        self.dropout3 = nn.Dropout(dp)

        self.bn2 = nn.BatchNorm1d(d_hidden)


    def forward(self, x):

        x = self.embed(x)

        x = x.mean(dim=1)

        x = self.dropout1(x)

        x = self.bn1(x)

        x = self.fc1(x)
        
        x = self.non_linear_function(x)


        x = self.dropout2(x)

        x = self.bn2(x)

        x = self.fc2(x)

        return x

In [45]:
model = DeepAverageNetwork(n_embed=100_000, 
                           d_embed=len(embeddings[0]),
                           embed_weight= embeddings,
                           d_hidden=128, 
                           d_out=3, 
                           dp=0.22, 
                           )

In [28]:
def train(model, iterator, optimizer, criterion):
    
    losses = list()
    f1 = list()
    
    model.train()  

    for x, y in tqdm(iterator, total=len(iterator)):
        
        optimizer.zero_grad()   
        predictions = model(x.to(device)).squeeze() 
        loss = criterion(predictions, y.to(device))        
               
        
        loss.backward()
        optimizer.step()     
        preds = predictions.detach().cpu().numpy().argmax(1).tolist()
        y_true = y.tolist()
        
        losses.append(loss.item())
        f1.append(f1_score(y_true, preds, average="weighted"))
        
    return np.mean(f1), np.mean(losses)

def evaluate(model, iterator, criterion):
    
    losses = list()
    f1 = list()
    
    model.eval()  
    with torch.no_grad():
        for x, y in tqdm(iterator, total=len(iterator)):   

            predictions = model(x.to(device)).squeeze()  
            loss = criterion(predictions, y.to(device))        
            preds = predictions.detach().cpu().numpy().argmax(1).tolist()
            y_true = y.tolist()
            

            losses.append(loss.item())  
            f1.append(f1_score(y_true, preds, average="weighted"))
            
    return np.mean(f1), np.mean(losses) 

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [46]:
Epochs = 10

losses = list()
loss_on_eval = list()

f1 = list()
f1_on_eval = list()

for n_epoch in range(Epochs):
    print(n_epoch)
    f1_train, loss_train = train(model=model, iterator=train_loader, optimizer=optimizer, criterion=criterion)
    f1.append(f1_train)
    losses.append(loss_train)
    f1_valid, loss_valid = evaluate(model=model, iterator=valid_loader, criterion=criterion)
    f1_on_eval.append(f1_valid)
    loss_on_eval.append(loss_valid)


    print('Loss_on_train:{lt}, F1_train:{ft},\nLoss_on_valid:{lv}, F1_valid: {fv}'.format(lt = loss_train, ft = f1_train, lv = loss_valid, fv = f1_valid))
    print('---')

0




  0%|          | 0/357 [00:00<?, ?it/s]

  0%|          | 1/357 [00:00<04:33,  1.30it/s]

  1%|          | 2/357 [00:01<03:22,  1.76it/s]

  1%|          | 3/357 [00:01<02:55,  2.01it/s]

  1%|          | 4/357 [00:02<02:45,  2.13it/s]

  1%|▏         | 5/357 [00:02<02:37,  2.23it/s]

  2%|▏         | 6/357 [00:02<02:32,  2.30it/s]

  2%|▏         | 7/357 [00:03<02:28,  2.36it/s]

  2%|▏         | 8/357 [00:03<02:27,  2.37it/s]

  3%|▎         | 9/357 [00:04<02:26,  2.38it/s]

  3%|▎         | 10/357 [00:04<02:24,  2.41it/s]

  3%|▎         | 11/357 [00:04<02:23,  2.41it/s]

  3%|▎         | 12/357 [00:05<02:22,  2.42it/s]

  4%|▎         | 13/357 [00:05<02:21,  2.42it/s]

  4%|▍         | 14/357 [00:06<02:21,  2.42it/s]

  4%|▍         | 15/357 [00:06<02:20,  2.43it/s]

  4%|▍         | 16/357 [00:06<02:20,  2.44it/s]

  5%|▍         | 17/357 [00:07<02:20,  2.42it/s]

  5%|▌         | 18/357 [00:07<02:20,  2.42it/s]

  5%|▌         | 19/357 [00:08<02:19,  2.42it/s]

  6%|▌         |

Loss_on_train:1.2128521401007302, F1_train:0.34632175376885577,
Loss_on_valid:1.1231376305222511, F1_valid: 0.3191235868843091
---
1




  0%|          | 0/357 [00:00<?, ?it/s]

  0%|          | 1/357 [00:00<02:29,  2.39it/s]

  1%|          | 2/357 [00:00<02:31,  2.35it/s]

  1%|          | 3/357 [00:01<02:29,  2.36it/s]

  1%|          | 4/357 [00:01<02:28,  2.38it/s]

  1%|▏         | 5/357 [00:02<02:28,  2.38it/s]

  2%|▏         | 6/357 [00:02<02:26,  2.40it/s]

  2%|▏         | 7/357 [00:02<02:25,  2.41it/s]

  2%|▏         | 8/357 [00:03<02:25,  2.40it/s]

  3%|▎         | 9/357 [00:03<02:24,  2.41it/s]

  3%|▎         | 10/357 [00:04<02:23,  2.43it/s]

  3%|▎         | 11/357 [00:04<02:22,  2.42it/s]

  3%|▎         | 12/357 [00:04<02:21,  2.43it/s]

  4%|▎         | 13/357 [00:05<02:23,  2.40it/s]

  4%|▍         | 14/357 [00:05<02:22,  2.41it/s]

  4%|▍         | 15/357 [00:06<02:21,  2.41it/s]

  4%|▍         | 16/357 [00:06<02:21,  2.41it/s]

  5%|▍         | 17/357 [00:07<02:20,  2.43it/s]

  5%|▌         | 18/357 [00:07<02:20,  2.41it/s]

  5%|▌         | 19/357 [00:07<02:19,  2.42it/s]

  6%|▌         |

Loss_on_train:1.2134642320520737, F1_train:0.3478169018512724,
Loss_on_valid:1.1232090070843697, F1_valid: 0.313412561880935
---
2


...

Loss_on_train:1.2134359831235655, F1_train:0.3429304284682338,
Loss_on_valid:1.1231357529759407, F1_valid: 0.3190517179470804
---
8




  0%|          | 0/357 [00:00<?, ?it/s]

  0%|          | 1/357 [00:00<02:53,  2.05it/s]

  1%|          | 2/357 [00:01<02:58,  1.99it/s]

  1%|          | 3/357 [00:01<03:00,  1.96it/s]

  1%|          | 4/357 [00:02<02:55,  2.01it/s]

  1%|▏         | 5/357 [00:02<02:53,  2.03it/s]

  2%|▏         | 6/357 [00:02<02:49,  2.07it/s]

  2%|▏         | 7/357 [00:03<02:45,  2.11it/s]

  2%|▏         | 8/357 [00:03<02:42,  2.15it/s]

  3%|▎         | 9/357 [00:04<02:42,  2.14it/s]

  3%|▎         | 10/357 [00:04<02:40,  2.17it/s]

  3%|▎         | 11/357 [00:05<02:39,  2.17it/s]

  3%|▎         | 12/357 [00:05<02:36,  2.20it/s]

  4%|▎         | 13/357 [00:06<02:37,  2.19it/s]

  4%|▍         | 14/357 [00:06<02:37,  2.18it/s]

  4%|▍         | 15/357 [00:07<02:36,  2.19it/s]

  4%|▍         | 16/357 [00:07<02:35,  2.20it/s]

  5%|▍         | 17/357 [00:07<02:36,  2.18it/s]

  5%|▌         | 18/357 [00:08<02:38,  2.14it/s]

  5%|▌         | 19/357 [00:08<02:36,  2.16it/s]

  6%|▌         |

Loss_on_train:1.2127662660027085, F1_train:0.34475448688496096,
Loss_on_valid:1.1285557076334953, F1_valid: 0.3029211593178772
---
9




  0%|          | 0/357 [00:00<?, ?it/s]

  0%|          | 1/357 [00:00<02:37,  2.25it/s]

  1%|          | 2/357 [00:00<02:39,  2.23it/s]

  1%|          | 3/357 [00:01<02:43,  2.17it/s]

  1%|          | 4/357 [00:01<02:43,  2.16it/s]

  1%|▏         | 5/357 [00:02<02:41,  2.18it/s]

  2%|▏         | 6/357 [00:02<02:38,  2.21it/s]

  2%|▏         | 7/357 [00:03<02:38,  2.21it/s]

  2%|▏         | 8/357 [00:03<02:36,  2.23it/s]

  3%|▎         | 9/357 [00:04<02:36,  2.22it/s]

  3%|▎         | 10/357 [00:04<02:35,  2.23it/s]

  3%|▎         | 11/357 [00:04<02:34,  2.24it/s]

  3%|▎         | 12/357 [00:05<02:34,  2.24it/s]

  4%|▎         | 13/357 [00:05<02:33,  2.24it/s]

  4%|▍         | 14/357 [00:06<02:35,  2.21it/s]

  4%|▍         | 15/357 [00:06<02:33,  2.22it/s]

  4%|▍         | 16/357 [00:07<02:33,  2.22it/s]

  5%|▍         | 17/357 [00:07<02:32,  2.23it/s]

  5%|▌         | 18/357 [00:08<02:32,  2.23it/s]

  5%|▌         | 19/357 [00:08<02:31,  2.24it/s]

  6%|▌         |

Loss_on_train:1.2103326778118015, F1_train:0.3433361661137654,
Loss_on_valid:1.125163808465004, F1_valid: 0.30638698333971887
---


# **Результат довольно неплоохй. Метрика f1 weighted подошла хорошо, так как она нужна, если существует дисбаланс классов. Это как раз наш случай.**